### Import needed libraries

In [ ]:
# library provided by EPOCH for reading .sdf output files into Python
import sdf_helper as sh

# python plotting library similar to MATLAB(TM)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# signal processing, needed for convolution
from scipy import signal

# Python array manipulation
import numpy as np

# various math functions 
import math as m

In [ ]:
# show plots in the notebook
%matplotlib inline

### Define useful functions

In [ ]:
from collections import OrderedDict

In [ ]:
def list_sdf_variables(data):
    r"""Lists all the quantities from the .sdf file.
    
    Parameters
    ----------
    data : ``sdf.Blocklist``
        The results of calling sdf.read on an .sdf file.
    """
    dct = data.__dict__
    for key in sorted(dct):
        try:
            val = dct[key]
            print('{} {} {}'.format(key, type(val),
                  np.array2string(np.array(val.dims), separator=', ')))
        except:
            pass

bash console commands can be run inside the notebook, eg.

The following code will download the data needed for this notebook automatically using `curl`. It may take some time (the archive is 1.2 GB), so please wait when the kernel is busy. You will need to set `download_datasets` to `True` before using it.

In [ ]:
download_datasets = False
if download_datasets:
    !curl -sSO https://ndownloader.figshare.com/articles/5545165/versions/1
    print ("Downloaded the EPOCH data from figshare.")
    !unzip 1 
    
    print ("All done!")

In [ ]:
# these are the .sdf files used in the notebook. they must be present in the same folder when the notebook is run
!ls -lsa *.sdf

In [ ]:
# these are the corresponding EPOCH input decks
!ls -lsa *.deck

# 1D case

In [ ]:
!cat 1dinput.deck

For the 1D case, we just use `numpy` and `matplolib` for low-level plotting and data analysis.

In [ ]:
# load the sdf file produced by EPOCH
fname = '1d.sdf'
data_1d = sh.getdata(fname);

In [ ]:
sh.list_variables(data_1d)

In [ ]:
# assign separate variables to the various quantities we want to look at
grid = data_1d.Grid_Grid_mid
nele = data_1d.Derived_Number_Density_ele
ex = data_1d.Electric_Field_Ex
ey = data_1d.Electric_Field_Ey

In [ ]:
sh.plot_auto(nele)

In [ ]:
# the .data attribute contains the raw data we need to plot
(x,) = grid.data
# convert to micrometers
x = x*1e6

rho = nele.data

In [ ]:
# define size of Gaussian kernel used for noise reduction
kern_size = 100
rho_smooth = smooth(rho, 1, [kern_size])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,3])

axes[0].plot(x, rho)
axes[1].plot(x[kern_size:-kern_size], rho_smooth)

axes[0].set_title('raw data')
axes[1].set_title('after convolution with Gaussian kernel')

for ax in axes:
    # the labels and units were contained in the .sdf file
    ax.set_xlabel(nele.grid_mid.labels[0] + r' $(\mu m)$', labelpad=-1)
    ax.set_ylabel(nele.name + r' $(' + nele.units + r')$');

plt.tight_layout(h_pad=1)

# saves the generated figure to disk in the same folder that the notebook is ran from
fig.savefig('1d_rho.png')

In [ ]:
sh.plot_auto(ex)

In [ ]:
sh.plot_auto(ey)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[14,3])

for ax, ef in zip(axes, (ex.data, ey.data)):
    ax.plot(x, ef)

for ax, electric in zip(axes, (ex, ey)):
    ax.set_xlabel(electric.grid_mid.labels[0] + r' $(\mu m)$', labelpad=-1)
    ax.set_ylabel(electric.name + r' $(' + electric.units + r')$');

fig.suptitle('components of the electric field')
plt.tight_layout(h_pad=1)